In [1]:
import requests

from bs4 import BeautifulSoup
import pandas as pd
import re

acled = pd.read_csv("/Users/jonathanold/Library/CloudStorage/GoogleDrive-jonathan_old@berkeley.edu/My Drive/_Berkeley Research/Reservations and Conflict/Data/ACLED/acled_india_csv.csv", sep=";")

acled['riots'] =       (acled['event_type'] == 'Riots').astype(int)
acled['protests'] =    (acled['event_type'] == 'Protests').astype(int)
acled['strat_dev'] =   (acled['event_type'] == 'Strategic developments').astype(int)
acled['viol_civil'] =  (acled['event_type'] == 'Violence against civilians').astype(int)
acled['explosions'] =  (acled['event_type'] == 'Explosions/Remote violence').astype(int)
acled['battles'] =     (acled['event_type'] == 'Battles').astype(int)

acled['s_violent_protest'] =     (acled['sub_event_type'] == 'Violent demonstration').astype(int)
acled['s_peaceful_protest'] =     (acled['sub_event_type'] == 'Peaceful protest').astype(int)
acled['s_mob_violence'] =     (acled['sub_event_type'] == 'Mob violence').astype(int)
acled['s_protest_intervention'] =     (acled['sub_event_type'] == 'Protest with intervention').astype(int)
acled['s_armed_clash'] =     (acled['sub_event_type'] == 'Armed clash').astype(int)
acled['s_disr_weaponuse'] =     (acled['sub_event_type'] == 'Disrupted weapons use').astype(int)
acled['s_attack'] =     (acled['sub_event_type'] == 'Attack').astype(int)
acled['s_grenade'] =     (acled['sub_event_type'] == 'Grenade').astype(int)
acled['s_looting'] =     (acled['sub_event_type'] == 'Looting/property destruction').astype(int)
acled['s_change_group'] =     (acled['sub_event_type'] == 'Change to group/activity').astype(int)
acled['s_sexual_violence'] =     (acled['sub_event_type'] == 'Sexual violence').astype(int)
acled['s_remote_expl'] =     (acled['sub_event_type'] == 'Remote explosive/landmine/IED').astype(int)
acled['s_abduction'] =     (acled['sub_event_type'] == 'Abduction/forced disappearance').astype(int)
acled['s_arrest'] =     (acled['sub_event_type'] == 'Arrests').astype(int)
acled['s_shelling'] =     (acled['sub_event_type'] == 'Shelling/artillery/missile attack').astype(int)
acled['s_excessive_force'] =     (acled['sub_event_type'] == 'Excessive force against protesters').astype(int)
acled['s_other'] =     (acled['sub_event_type'] == 'Other').astype(int)
acled['s_agreement'] =     (acled['sub_event_type'] == 'Agreement').astype(int)
acled['s_drone'] =     (acled['sub_event_type'] == 'Air/drone strike').astype(int)
acled['s_suicide_bomb'] =     (acled['sub_event_type'] == 'Suicide bomb').astype(int)
acled['s_hq_base'] =     (acled['sub_event_type'] == 'Headquarters or base established').astype(int)
acled['s_transfer_of_territory'] =     (acled['sub_event_type'] == 'Non-violent transfer of territory').astype(int)
acled['s_gov_regains_territory'] =     (acled['sub_event_type'] == 'Government regains territory').astype(int)

acled['description'] = acled['notes']
acled.drop(columns=['notes','iso','region','country','iso3'], inplace=True)


variables = [
    'riots', 'protests', 'strat_dev', 'viol_civil', 'explosions', 'battles',
    's_violent_protest', 's_peaceful_protest', 's_mob_violence', 's_protest_intervention',
    's_armed_clash', 's_disr_weaponuse', 's_attack', 's_grenade', 's_looting',
    's_change_group', 's_sexual_violence', 's_remote_expl', 's_abduction', 's_arrest',
    's_shelling', 's_excessive_force', 's_other', 's_agreement', 's_drone', 's_suicide_bomb',
    's_hq_base', 's_transfer_of_territory', 's_gov_regains_territory'
]

for var in variables:
    if (acled[var] != 0).sum() < 100:
        acled.drop(columns=[var], inplace=True)


In [2]:
print(len(acled))
acled['caste'] = 0
acled['communal'] = 0

# Caste
acled.loc[acled['actor1'].str.contains('caste', case=False, na=False), 'caste'] = 1
acled.loc[acled['assoc_actor_1'].str.contains('caste', case=False, na=False), 'caste'] = 1
acled.loc[acled['actor2'].str.contains('caste', case=False, na=False), 'caste'] = 1
acled.loc[acled['assoc_actor_2'].str.contains('caste', case=False, na=False), 'caste'] = 1

acled.loc[acled['description'].str.contains(' caste ', case=False, na=False), 'caste'] = 1



acled.loc[acled['actor1'].str.contains('communal militia', case=False, na=False), 'communal'] = 1
acled.loc[acled['actor2'].str.contains('communal militia', case=False, na=False), 'communal'] = 1
acled.loc[acled['assoc_actor_1'].str.contains('communal militia', case=False, na=False), 'communal'] = 1
acled.loc[acled['assoc_actor_2'].str.contains('communal militia', case=False, na=False), 'communal'] = 1


74902


In [20]:
keywords = [#"caste", "schedule trib", "schedule cast", "brahmin", "untouch", "obc", "dalit", "reserv", 
            "jat", "jaat", " sc ", "discrim", "(sc)", "category", "quota"]
stopwords = ["cprf reserve", "reserve engineering", "reserve guard", "tiger reserve", "elephant reserve", "reserve police", "reserve forest", "reserve force", "reservoire", "release reserves", "reserve group", "reserve bank"]

# Generate acled['des2'] that is acled['description'] with all stopwords removed
acled['des2'] = acled['description'].str.replace('|'.join(stopwords), '', case=False, regex=True)


for keyword in keywords:
    var_name = f'var_{keyword}'
    acled[var_name] = acled['des2'].str.contains(keyword, case=False, na=False).astype(int) 
                       
# Gen df, adf, that is these rows from acled: description, keywords
adf = acled[[f'var_{keyword}' for keyword in keywords]]
# Add 'description' to at the beginning of adf
adf.insert(0, 'description', acled['description'])


# Generate the cross-correlation table
correlation_table = acled[[f'var_{keyword}' for keyword in keywords]].corr().round(4)

print(correlation_table)

/var/folders/yz/pz74ttxj0g9ggdqz9nnmm5q00000gn/T/ipykernel_22543/602885236.py:11: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  acled[var_name] = acled['des2'].str.contains(keyword, case=False, na=False).astype(int)


              var_jat  var_jaat  var_ sc   var_discrim  var_(sc)  \
var_jat        1.0000    0.0134   -0.0031      -0.0050   -0.0213   
var_jaat       0.0134    1.0000   -0.0003      -0.0005    0.0011   
var_ sc       -0.0031   -0.0003    1.0000      -0.0017    0.0925   
var_discrim   -0.0050   -0.0005   -0.0017       1.0000    0.1472   
var_(sc)      -0.0213    0.0011    0.0925       0.1472    1.0000   
var_category   0.2332   -0.0006    0.0391      -0.0031    0.0342   
var_quota      0.3058   -0.0005    0.0133      -0.0028   -0.0052   

              var_category  var_quota  
var_jat             0.2332     0.3058  
var_jaat           -0.0006    -0.0005  
var_ sc             0.0391     0.0133  
var_discrim        -0.0031    -0.0028  
var_(sc)            0.0342    -0.0052  
var_category        1.0000     0.0248  
var_quota           0.0248     1.0000  


In [4]:
from sklearn.decomposition import PCA
import numpy as np

# List of generated variable names
variables = [f'var_{keyword}' for keyword in keywords]

# Perform PCA
pca = PCA(n_components=4)
principal_components = pca.fit_transform(acled[variables])

# Get the loadings (correlations of each variable with the first principal component)
loadings = pca.components_[0]
loadings2 = pca.components_[1]
loadings3 = pca.components_[2]
loadings4 = pca.components_[3]

# Create a DataFrame to show the correlations
correlation_with_first_component = pd.DataFrame({'Variable': variables, 'Correlation 1': loadings,
                                                 'Correlation 2': loadings2,
                                                 'Correlation 3': loadings3,
                                                 'Correlation 4': loadings4})

print(correlation_with_first_component)

             Variable  Correlation 1  Correlation 2  Correlation 3  \
0           var_caste       0.047746       0.256864       0.091127   
1   var_schedule trib       0.003451      -0.000797       0.001156   
2   var_schedule cast       0.000917       0.001896       0.001790   
3         var_brahmin       0.001534       0.004264       0.001440   
4         var_untouch       0.000383       0.004604      -0.000548   
5         var_forward      -0.000437      -0.000216       0.004624   
6             var_obc       0.004555       0.009095       0.186860   
7           var_dalit       0.048539       0.962667      -0.082050   
8          var_reserv       0.003422       0.056238       0.895838   
9             var_jat      -0.006969       0.008122       0.338501   
10           var_jaat       0.000037       0.000449       0.000117   
11           var_ sc        0.009768       0.003681       0.003378   
12        var_discrim       0.025186       0.018250       0.000532   
13           var_(sc

In [5]:

satp_data = pd.read_csv("/Users/jonathanold/Library/CloudStorage/GoogleDrive-jonathan_old@berkeley.edu/My Drive/_Berkeley Research/Reservations and Conflict/Data/_gen/coding_events/satp_training_data_edited.csv", encoding='utf-8')

acled_plus_satp = pd.concat([acled, satp_data])
acled_plus_satp.to_csv("/Users/jonathanold/Library/CloudStorage/GoogleDrive-jonathan_old@berkeley.edu/My Drive/_Berkeley Research/Reservations and Conflict/Data/_gen/coding_events/acled_plus_satp.csv")











/var/folders/yz/pz74ttxj0g9ggdqz9nnmm5q00000gn/T/ipykernel_22543/2789530295.py:1: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  satp_data = pd.read_csv("/Users/jonathanold/Library/CloudStorage/GoogleDrive-jonathan_old@berkeley.edu/My Drive/_Berkeley Research/Reservations and Conflict/Data/_gen/coding_events/satp_training_data_edited.csv", encoding='utf-8')


KeyboardInterrupt: 

In [ ]:



unique_values = acled['assoc_actor_2'].unique()

# Display the unique values
print(unique_values)

[nan 'BJP: Bharatiya Janata Party'
 'Former Government of India (2014-); BJP: Bharatiya Janata Party'
 'Health Workers (India)' 'Government of India (2014-)'
 'Police Forces of India (2014-)' 'HM: Hizb-ul-Mujahideen'
 'Women (India); Government of India (2014-)'
 'YSRCP: Yuvajana, Sramika, Rythu Congress Party' 'Labour Group (India)'
 'Chapunna Communal Group (India)'
 'Civilians (India); TMC: Trinamool Congress Party; Government of India (2014-)'
 'Chamraua Communal Group (India)'
 'Hindu Group (India); Muslim Group (India)' 'Rioters (India)'
 'Police Forces of India (2014-) Central Reserve Police Force; Police Forces of India (2014-) Special Operations Group'
 'RJD: Rashtriya Janata Dal' 'Health Workers (India); Women (India)'
 'Police Forces of India (2014-) Rapid Action Force'
 'CPI(M): Communist Party of India (Marxist); INC: Indian National Congress'
 'Police Forces of India (2014-) Central Reserve Police Force'
 'Women (India)' 'Muslim Group (India)' 'Farmers (India)'
 'Dalit Ca

In [ ]:
descriptions = acled[acled['caste'] == 1]['description']
print(descriptions)


341      On 20 May 2020, a clash occurred between upper...
573      On 13 May 2020, a tribal man and his two sons ...
720      On 9 May 2020, members of the Koli caste surro...
1586     On 5 Apr 2020, at least six people from domina...
2024     On Mar 16 2020, members of the All Assam Sched...
                               ...                        
74413    On 20 Jan, a group of caste Hindus attacked a ...
74473    In Kolkata, Jadavpur University became the pri...
74558    On 14 Jan 2016, in Chennai area (Chennai, TN),...
74561    On 14 Jan 2016, in Chennai area (Chennai, TN),...
74616    Dalit Indian Administrative Services officers ...
Name: description, Length: 473, dtype: object


In [ ]:

descriptions2 = acled[acled['communal'] == 1]['description']
print(descriptions2)